# Xns11 Examples - Basic writing and modifying of cross sections

In [ ]:
from mikeio1d import Xns11
from mikeio1d.cross_sections import CrossSection
from mikeio1d.cross_sections import Marker
import numpy as np
import matplotlib.pyplot as plt

#### Create a custom cross section from raw x, z points.

In [ ]:
# Create some imaginary cross section data. You can use your own data here.
n_points = 11
x = np.linspace(0, 100, n_points)
z = 10**-3 * (x-50)**2
plt.plot(x, z);

In [ ]:
# Create a CrossSection object from the xz data. Every cross section must have a location id, chainage, and topo id.
location_id = "my_reach"
chainage = 100.0
topo_id = "my_topo"

xs = CrossSection.from_xz(x, z, location_id, chainage, topo_id)
xs

In [ ]:
# Plot the cross section to verify that it looks correct. 
# Notice that default markers are placed (to disable, use default_markers=False when calling create_from_xz).
xs.plot()

In [ ]:
# Explore the raw data of the cross section to verify it looks correct.
xs.raw

In [ ]:
# Similarly, look at the processed data.
xs.processed

#### Create an Xns11 file and add a cross section.

In [ ]:
# Create an empty Xns11 object.
xns = Xns11()
xns.info()

In [ ]:
# Add a cross section to the Xns11 object.
xns.add_xsection(xs)
xns.info()

In [ ]:
# Save the Xns11 object to a file.
xns.write("my_xns11.xns11")

In [ ]:
# Open the file and read the cross section from it as a test
xns2 = Xns11("my_xns11.xns11")
xs = xns2.xsections.sel(location_id="my_reach", chainage=100, topo_id="my_topo")
xs.plot()

#### Extract a subset of cross sections and save to another xns11 file.

In [ ]:
# Open an existing Xns11 file and extract a slice from it
xns = Xns11("../tests/testdata/mikep_cs_demo.xns11")
sections = xns.xsections['basin_left1']
sections.to_dataframe()

In [ ]:
# Create a new Xns11 object based on these cross sections.
xns = Xns11.from_cross_section_collection(sections)
xns.info()

In [ ]:
# Save the new Xns11 object to a file
xns.write("basin_left1_sections.xns11")

#### Update an existing cross section's raw data.

In [ ]:
# Get the current raw data as a DataFrame to use for further processing.
raw_modified = xns.xsections['basin_left1','122.042','1'].raw
raw_modified

In [ ]:
# Add 1000 to all 'z' elevations.
raw_modified.z += 1000
raw_modified

In [ ]:
# Assign the modified raw data back to the cross section.
xns.xsections['basin_left1','122.042','1'].raw = raw_modified

In [ ]:
# Write the changes to the file, otherwise they will be lost.
xns.write()

In [ ]:
# Confirm the raw data was saved.
xns = Xns11("basin_left1_sections.xns11")
xns.xsections['basin_left1','122.042','1'].raw

In [ ]:
# Update the coordinate markers. Start by getting the current markers.
xns.xsections['basin_left1','122.042','1'].plot()
df = xns.xsections['basin_left1','122.042','1'].markers
df

In [ ]:
# Set the left and right low flow banks markers, and plot the cross section to verify the changes.
xns.xsections['basin_left1','122.042','1'].set_marker(Marker.LEFT_LOW_FLOW_BANK, x=70)
xns.xsections['basin_left1','122.042','1'].set_marker(Marker.RIGHT_LOW_FLOW_BANK, x=74)
xns.xsections['basin_left1','122.042','1'].plot(with_marker_labels=False)
xns.xsections['basin_left1','122.042','1'].markers

In [ ]:
# Add a user defined marker.
xns.xsections['basin_left1','122.042','1'].set_marker(42, x=150)
xns.xsections['basin_left1','122.042','1'].plot(with_marker_labels=False)
xns.xsections['basin_left1','122.042','1'].markers

In [ ]:
# You can also modify the markers by directly modifying the DataFrame and reassigning it.
df = xns.xsections['basin_left1','122.042','1'].markers
df = df[~df.marker.isin([Marker.LEFT_LEVEE_BANK.value, Marker.RIGHT_LEVEE_BANK.value])]
xns.xsections['basin_left1','122.042','1'].markers = df
xns.xsections['basin_left1','122.042','1'].recompute_processed()
xns.xsections['basin_left1','122.042','1'].plot();

In [ ]:
# Save the changes to the file.
xns.write()

In [ ]:
# Similarly, you can update the raw data by reassigning a modified DataFrame.
raw_modified = xns.xsections['basin_left1','122.042','1'].raw
raw_modified.resistance = 50
xns.xsections['basin_left1','122.042','1'].raw = raw_modified
xns.xsections['basin_left1','122.042','1'].raw

In [ ]:
xns.write()

In [ ]:
xns.xsections['basin_left1','122.042','1'].resistance_distribution

In [ ]:
xns.xsections['basin_left1','122.042','1'].resistance_distribution = 1
xns.xsections['basin_left1','122.042','1'].raw

In [ ]:
xns.xsections['basin_left1','122.042','1'].resistance_left_high_flow

In [ ]:
xns.xsections['basin_left1','122.042','1'].resistance_distribution = 1
xns.xsections['basin_left1','122.042','1'].raw

In [ ]:
xns.xsections['basin_left1','122.042','1'].resistance_left_high_flow

In [ ]:
xns.xsections['basin_left1','122.042','1'].resistance_distribution = 0
xns.xsections['basin_left1','122.042','1'].resistance_left_high_flow = 25
xns.xsections['basin_left1','122.042','1'].resistance_low_flow = 5
xns.xsections['basin_left1','122.042','1'].resistance_right_low_flow = 30
xns.xsections['basin_left1','122.042','1'].raw.iloc[20:70]

In [ ]:
xns.xsections['basin_left1','122.042','1'].plot()

#### Clean up files we created.

In [ ]:
import os
os.remove("my_xns11.xns11")
os.remove("basin_left1_sections.xns11")